# Equity Classification Using Financial Ratios

## Complete Machine Learning Pipeline

This notebook contains the complete analysis pipeline:
1. Data Loading & Validation
2. Exploratory Data Analysis (EDA)
3. Data Preprocessing & Missing Value Imputation
4. Financial Ratio Computation
5. Correlation Analysis & Feature Selection
6. Model Training & Evaluation

---


## Setup & Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Model Selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import (classification_report, confusion_matrix, 
                             accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, roc_curve)

# Settings
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

print(" All libraries imported successfully!")

---
## Part 1: Data Loading & Validation

Load your existing financial statement data.

### Expected Data Format:

Your CSV/Excel file should contain the following columns:

**Identification Columns:**
- `Company`: Company name or ticker
- `Sector`: Industry sector (optional)
- `Year`: Fiscal year
- `Quarter`: Fiscal quarter (optional)

**Income Statement:**
- `Revenue`: Total revenue/sales
- `COGS`: Cost of goods sold
- `Gross_Profit`: Revenue - COGS
- `Operating_Expenses`: Operating expenses
- `EBIT`: Earnings before interest and tax
- `Interest_Expense`: Interest payments
- `EBT`: Earnings before tax
- `Tax`: Tax expense
- `Net_Income`: Net profit

**Balance Sheet:**
- `Current_Assets`: Current assets
- `Cash`: Cash and equivalents
- `Accounts_Receivable`: Receivables
- `Inventory`: Inventory
- `Fixed_Assets`: Non-current assets
- `Total_Assets`: Total assets
- `Current_Liabilities`: Current liabilities
- `Accounts_Payable`: Payables
- `Short_Term_Debt`: Short-term debt
- `Long_Term_Debt`: Long-term debt
- `Total_Liabilities`: Total liabilities
- `Shareholders_Equity`: Equity

**Cash Flow Statement:**
- `Operating_Cash_Flow`: Cash from operations
- `CapEx`: Capital expenditure
- `Free_Cash_Flow`: OCF - CapEx

**Market Data:**
- `Shares_Outstanding`: Number of shares
- `Stock_Price`: Share price
- `Market_Cap`: Market capitalization
- `EPS`: Earnings per share
- `Dividend_Per_Share`: Dividend per share (optional)

**Target Variable:**
- `Investment_Quality`: 'Good' or 'Bad' (your classification label)


In [ ]:
# ========================================
# LOAD YOUR DATA HERE
# ========================================

# Option 1: Load from CSV
file_path = 'your_financial_data.csv'  # ← Change this to your file path
df_raw = pd.read_csv(file_path)

# Option 2: Load from Excel
# file_path = 'your_financial_data.xlsx'
# df_raw = pd.read_excel(file_path, sheet_name='Sheet1')

# Option 3: Load from multiple files (if you have separate files for each company/year)
# import glob
# files = glob.glob('financial_data/*.csv')
# df_raw = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

print(f" Data loaded successfully!")
print(f"  Shape: {df_raw.shape}")
print(f"  Companies: {df_raw['Company'].nunique() if 'Company' in df_raw.columns else 'N/A'}")
print(f"  Years: {df_raw['Year'].min() if 'Year' in df_raw.columns else 'N/A'} - {df_raw['Year'].max() if 'Year' in df_raw.columns else 'N/A'}")

### 1.1 Data Validation

In [ ]:
# Check for required columns
required_columns = [
    'Company', 'Year', 'Revenue', 'Net_Income', 'Total_Assets', 
    'Total_Liabilities', 'Shareholders_Equity', 'Current_Assets', 
    'Current_Liabilities', 'Investment_Quality'
]

missing_cols = [col for col in required_columns if col not in df_raw.columns]

if missing_cols:
    print("WARNING: Missing required columns:")
    for col in missing_cols:
        print(f"   - {col}")
    print("\n   Please ensure your data contains these columns.")
else:
    print(" All required columns present")

# Display available columns
print(f"\nAvailable columns ({len(df_raw.columns)}):")
for i, col in enumerate(df_raw.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Display first few rows
print("First 5 rows of data:")
df_raw.head()

In [ ]:
# Display last few rows
print("Last 5 rows of data:")
df_raw.tail()

In [ ]:
# Data types and basic info
print("Dataset Information:")
df_raw.info()

---
## Part 2: Exploratory Data Analysis (EDA)

### 2.1 Data Overview

In [ ]:
# Basic statistics
print("Dataset Summary:")
print(f"Total Records: {len(df_raw):,}")
print(f"Unique Companies: {df_raw['Company'].nunique() if 'Company' in df_raw.columns else 'N/A'}")
print(f"Time Period: {df_raw['Year'].min() if 'Year' in df_raw.columns else 'N/A'} - {df_raw['Year'].max() if 'Year' in df_raw.columns else 'N/A'}")
if 'Sector' in df_raw.columns:
    print(f"Sectors: {df_raw['Sector'].nunique()}")
print(f"Columns: {len(df_raw.columns)}")

### 2.2 Missing Values Analysis

In [ ]:
# Check missing values
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Percentage': missing_pct.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print("Missing Values Summary:")
    print(missing_df.to_string(index=False))
    
    # Visualize missing values
    plt.figure(figsize=(12, 6))
    plt.barh(missing_df['Column'], missing_df['Missing_Count'])
    plt.xlabel('Number of Missing Values')
    plt.title('Missing Values by Column')
    plt.tight_layout()
    plt.show()
else:
    print(" No missing values found in the dataset")

### 2.3 Target Variable Distribution

In [ ]:
# Check if Investment_Quality column exists
if 'Investment_Quality' in df_raw.columns:
    # Target variable distribution
    print("Investment Quality Distribution:")
    print(df_raw['Investment_Quality'].value_counts())
    print("\nPercentages:")
    print(df_raw['Investment_Quality'].value_counts(normalize=True) * 100)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count plot
    df_raw['Investment_Quality'].value_counts().plot(kind='bar', ax=axes[0], color=['#51cf66', '#ff6b6b'])
    axes[0].set_title('Investment Quality Distribution (Count)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Investment Quality', fontsize=12)
    axes[0].set_ylabel('Count', fontsize=12)
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Pie chart
    df_raw['Investment_Quality'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                                                      colors=['#51cf66', '#ff6b6b'])
    axes[1].set_title('Investment Quality Distribution (%)', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()
else:
    print("  WARNING: 'Investment_Quality' column not found in dataset")
    print("   Please ensure your data has a target variable for classification")

### 2.4 Descriptive Statistics

In [ ]:
# Select key financial metrics that exist in the dataset
potential_metrics = ['Revenue', 'Net_Income', 'Total_Assets', 'Total_Liabilities', 
                     'Shareholders_Equity', 'Operating_Cash_Flow', 'EPS', 'Stock_Price',
                     'EBIT', 'Gross_Profit', 'Current_Assets', 'Current_Liabilities']

key_metrics = [col for col in potential_metrics if col in df_raw.columns]

if key_metrics:
    print("Key Financial Metrics - Summary Statistics:")
    display(df_raw[key_metrics].describe().round(2))
else:
    print("Key financial metrics not found. Showing all numerical columns:")
    display(df_raw.describe().round(2))

### 2.5 Distribution Analysis

In [ ]:
# Distribution plots for key metrics
metrics_to_plot = [col for col in ['Revenue', 'Net_Income', 'Total_Assets', 'EPS', 'Operating_Cash_Flow', 'Stock_Price'] 
                   if col in df_raw.columns]

if metrics_to_plot:
    n_plots = len(metrics_to_plot)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.ravel() if n_plots > 1 else [axes]
    
    for idx, metric in enumerate(metrics_to_plot):
        data = df_raw[metric].dropna()
        axes[idx].hist(data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        axes[idx].set_title(f'Distribution of {metric}', fontweight='bold')
        axes[idx].set_xlabel(metric)
        axes[idx].set_ylabel('Frequency')
        axes[idx].grid(axis='y', alpha=0.3)
        
        # Add statistics
        skew = data.skew()
        kurt = data.kurtosis()
        axes[idx].text(0.95, 0.95, f'Skew: {skew:.2f}\nKurt: {kurt:.2f}',
                      transform=axes[idx].transAxes,
                      verticalalignment='top',
                      horizontalalignment='right',
                      bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Hide unused subplots
    for idx in range(n_plots, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    print("No standard financial metrics found for distribution analysis")

### 2.6 Sector Analysis (if available)

In [ ]:
# Sector analysis if Sector column exists
if 'Sector' in df_raw.columns:
    print("Companies by Sector:")
    print(df_raw.groupby('Sector')['Company'].nunique().sort_values(ascending=False))
    
    if 'Investment_Quality' in df_raw.columns:
        print("\nInvestment Quality by Sector (%):")
        sector_quality = pd.crosstab(df_raw['Sector'], df_raw['Investment_Quality'], normalize='index') * 100
        print(sector_quality.round(1))
        
        # Visualize
        sector_quality.plot(kind='bar', figsize=(12, 6), stacked=False)
        plt.title('Investment Quality by Sector', fontsize=14, fontweight='bold')
        plt.xlabel('Sector', fontsize=12)
        plt.ylabel('Percentage', fontsize=12)
        plt.legend(title='Investment Quality')
        plt.xticks(rotation=45)
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print(" No 'Sector' column found. Skipping sector analysis.")

### 2.7 Correlation Analysis (Raw Metrics)

In [ ]:
# Select numerical columns for correlation
potential_corr_vars = ['Revenue', 'Gross_Profit', 'EBIT', 'Net_Income', 
                       'Total_Assets', 'Total_Liabilities', 'Shareholders_Equity',
                       'Operating_Cash_Flow', 'Free_Cash_Flow', 'EPS']

correlation_vars = [col for col in potential_corr_vars if col in df_raw.columns]

if len(correlation_vars) >= 3:
    # Calculate correlation matrix
    corr_matrix = df_raw[correlation_vars].corr()
    
    # Visualize correlation matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix of Financial Metrics', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated pairs
    high_corr = []
    for i in range(len(corr_matrix)):
        for j in range(i+1, len(corr_matrix)):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                high_corr.append({
                    'Variable 1': corr_matrix.index[i],
                    'Variable 2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
    
    if high_corr:
        high_corr_df = pd.DataFrame(high_corr).sort_values('Correlation', key=abs, ascending=False)
        print("\nHighly Correlated Pairs (|r| > 0.8):")
        print(high_corr_df.head(10).to_string(index=False))
else:
    print("Not enough numerical columns for correlation analysis")

---
## Part 3: Data Preprocessing & Missing Value Imputation

### 3.1 Handle Missing Values

In [ ]:
# Create a copy for processing
df = df_raw.copy()

# Check missing values before imputation
missing_before = df.isnull().sum().sum()
print(f"Missing values before imputation: {missing_before:,}")

if missing_before > 0:
    # Median imputation for numerical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # Exclude identifier columns from imputation
    exclude_cols = ['Year', 'Quarter'] if 'Quarter' in numeric_cols else ['Year']
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
    
    if numeric_cols:
        print(f"\nImputing {len(numeric_cols)} numerical columns...")
        imputer = SimpleImputer(strategy='median')
        df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
    
    missing_after = df.isnull().sum().sum()
    print(f"Missing values after imputation: {missing_after:,}")
    print(f" Imputed {missing_before - missing_after:,} missing values using median strategy")
else:
    print("No missing values to impute")

---
## Part 4: Financial Ratio Computation

We'll compute financial ratios across multiple categories:
1. Profitability Ratios
2. Liquidity Ratios
3. Leverage/Solvency Ratios
4. Efficiency/Activity Ratios
5. Valuation Ratios
6. Cash Flow Ratios
7. Growth Metrics

In [ ]:
def safe_divide(numerator, denominator, default=0):
    """Safely divide two series, handling division by zero"""
    result = numerator / denominator
    result = result.replace([np.inf, -np.inf], np.nan)
    return result.fillna(default)

print("Computing Financial Ratios...\n")

ratios = {}
computed_count = 0

# ====================
# 1. PROFITABILITY RATIOS
# ====================
print("1. Profitability Ratios")
profitability_count = 0

if 'Gross_Profit' in df.columns and 'Revenue' in df.columns:
    ratios['Gross_Profit_Margin'] = safe_divide(df['Gross_Profit'], df['Revenue']) * 100
    profitability_count += 1

if 'EBIT' in df.columns and 'Revenue' in df.columns:
    ratios['Operating_Profit_Margin'] = safe_divide(df['EBIT'], df['Revenue']) * 100
    profitability_count += 1

if 'Net_Income' in df.columns and 'Revenue' in df.columns:
    ratios['Net_Profit_Margin'] = safe_divide(df['Net_Income'], df['Revenue']) * 100
    profitability_count += 1

if 'Net_Income' in df.columns and 'Total_Assets' in df.columns:
    ratios['ROA'] = safe_divide(df['Net_Income'], df['Total_Assets']) * 100
    profitability_count += 1

if 'Net_Income' in df.columns and 'Shareholders_Equity' in df.columns:
    ratios['ROE'] = safe_divide(df['Net_Income'], df['Shareholders_Equity']) * 100
    profitability_count += 1

if 'Operating_Cash_Flow' in df.columns and 'Revenue' in df.columns:
    ratios['OCF_Margin'] = safe_divide(df['Operating_Cash_Flow'], df['Revenue']) * 100
    profitability_count += 1

print(f"    Computed {profitability_count} profitability ratios")
computed_count += profitability_count

# ====================
# 2. LIQUIDITY RATIOS
# ====================
print("2. Liquidity Ratios")
liquidity_count = 0

if 'Current_Assets' in df.columns and 'Current_Liabilities' in df.columns:
    ratios['Current_Ratio'] = safe_divide(df['Current_Assets'], df['Current_Liabilities'])
    liquidity_count += 1

if all(col in df.columns for col in ['Current_Assets', 'Inventory', 'Current_Liabilities']):
    ratios['Quick_Ratio'] = safe_divide(df['Current_Assets'] - df['Inventory'], df['Current_Liabilities'])
    liquidity_count += 1

if 'Cash' in df.columns and 'Current_Liabilities' in df.columns:
    ratios['Cash_Ratio'] = safe_divide(df['Cash'], df['Current_Liabilities'])
    liquidity_count += 1

print(f"    Computed {liquidity_count} liquidity ratios")
computed_count += liquidity_count

# ====================
# 3. LEVERAGE RATIOS
# ====================
print("3. Leverage/Solvency Ratios")
leverage_count = 0

if 'Total_Liabilities' in df.columns and 'Shareholders_Equity' in df.columns:
    ratios['Debt_to_Equity'] = safe_divide(df['Total_Liabilities'], df['Shareholders_Equity'])
    leverage_count += 1

if 'Total_Liabilities' in df.columns and 'Total_Assets' in df.columns:
    ratios['Debt_to_Assets'] = safe_divide(df['Total_Liabilities'], df['Total_Assets'])
    leverage_count += 1

if 'Shareholders_Equity' in df.columns and 'Total_Assets' in df.columns:
    ratios['Equity_Ratio'] = safe_divide(df['Shareholders_Equity'], df['Total_Assets'])
    leverage_count += 1

if 'EBIT' in df.columns and 'Interest_Expense' in df.columns:
    ratios['Interest_Coverage'] = safe_divide(df['EBIT'], df['Interest_Expense'])
    leverage_count += 1

print(f"    Computed {leverage_count} leverage ratios")
computed_count += leverage_count

# ====================
# 4. EFFICIENCY RATIOS
# ====================
print("4. Efficiency/Activity Ratios")
efficiency_count = 0

if 'Revenue' in df.columns and 'Total_Assets' in df.columns:
    ratios['Asset_Turnover'] = safe_divide(df['Revenue'], df['Total_Assets'])
    efficiency_count += 1

if 'COGS' in df.columns and 'Inventory' in df.columns:
    ratios['Inventory_Turnover'] = safe_divide(df['COGS'], df['Inventory'])
    efficiency_count += 1

if 'Revenue' in df.columns and 'Accounts_Receivable' in df.columns:
    ratios['Receivables_Turnover'] = safe_divide(df['Revenue'], df['Accounts_Receivable'])
    efficiency_count += 1

print(f"    Computed {efficiency_count} efficiency ratios")
computed_count += efficiency_count

# ====================
# 5. VALUATION RATIOS
# ====================
print("5. Valuation Ratios")
valuation_count = 0

if 'Stock_Price' in df.columns and 'EPS' in df.columns:
    ratios['PE_Ratio'] = safe_divide(df['Stock_Price'], df['EPS'])
    valuation_count += 1

if 'Market_Cap' in df.columns and 'Shareholders_Equity' in df.columns:
    ratios['PB_Ratio'] = safe_divide(df['Market_Cap'], df['Shareholders_Equity'])
    valuation_count += 1

if 'Dividend_Per_Share' in df.columns and 'Stock_Price' in df.columns:
    ratios['Dividend_Yield'] = safe_divide(df['Dividend_Per_Share'], df['Stock_Price']) * 100
    valuation_count += 1

if 'EPS' in df.columns and 'Stock_Price' in df.columns:
    ratios['Earnings_Yield'] = safe_divide(df['EPS'], df['Stock_Price']) * 100
    valuation_count += 1

print(f"    Computed {valuation_count} valuation ratios")
computed_count += valuation_count

# ====================
# 6. CASH FLOW RATIOS
# ====================
print("6. Cash Flow Ratios")
cashflow_count = 0

if 'Operating_Cash_Flow' in df.columns and 'Current_Liabilities' in df.columns:
    ratios['OCF_Ratio'] = safe_divide(df['Operating_Cash_Flow'], df['Current_Liabilities'])
    cashflow_count += 1

if 'Free_Cash_Flow' in df.columns and 'Shareholders_Equity' in df.columns:
    ratios['FCF_to_Equity'] = safe_divide(df['Free_Cash_Flow'], df['Shareholders_Equity'])
    cashflow_count += 1

if 'Operating_Cash_Flow' in df.columns and 'Total_Liabilities' in df.columns:
    ratios['CF_to_Debt'] = safe_divide(df['Operating_Cash_Flow'], df['Total_Liabilities'])
    cashflow_count += 1

print(f"    Computed {cashflow_count} cash flow ratios")
computed_count += cashflow_count

# ====================
# 7. GROWTH METRICS
# ====================
print("7. Growth Metrics (YoY)")
growth_count = 0

if 'Company' in df.columns and 'Year' in df.columns:
    df = df.sort_values(['Company', 'Year'])
    
    if 'Revenue' in df.columns:
        ratios['Revenue_Growth'] = df.groupby('Company')['Revenue'].pct_change() * 100
        growth_count += 1
    
    if 'Net_Income' in df.columns:
        ratios['Earnings_Growth'] = df.groupby('Company')['Net_Income'].pct_change() * 100
        growth_count += 1
    
    if 'Total_Assets' in df.columns:
        ratios['Asset_Growth'] = df.groupby('Company')['Total_Assets'].pct_change() * 100
        growth_count += 1

print(f"    Computed {growth_count} growth metrics")
computed_count += growth_count

# Combine ratios with original data
if ratios:
    ratios_df = pd.DataFrame(ratios)
    df_with_ratios = pd.concat([df, ratios_df], axis=1)
    
    print(f"\n Total ratios computed: {computed_count}")
    print(f" Dataset shape after adding ratios: {df_with_ratios.shape}")
else:
    print("\n WARNING: No ratios could be computed. Check if required columns exist.")
    df_with_ratios = df.copy()

### 4.1 Handle Infinite and Extreme Values

In [ ]:
# Replace infinite values with NaN
numeric_df = df_with_ratios.select_dtypes(include=[np.number])
inf_count_before = np.isinf(numeric_df).sum().sum()
df_with_ratios = df_with_ratios.replace([np.inf, -np.inf], np.nan)

if inf_count_before > 0:
    print(f" Replaced {inf_count_before:,} infinite values with NaN")

# Cap extreme values at 1st and 99th percentile for ratios
ratio_columns = list(ratios.keys())
capped_count = 0

if ratio_columns:
    for col in ratio_columns:
        if df_with_ratios[col].dtype in ['float64', 'int64']:
            p1 = df_with_ratios[col].quantile(0.01)
            p99 = df_with_ratios[col].quantile(0.99)
            outliers = ((df_with_ratios[col] < p1) | (df_with_ratios[col] > p99)).sum()
            
            if outliers > 0:
                df_with_ratios[col] = df_with_ratios[col].clip(lower=p1, upper=p99)
                capped_count += outliers
    
    if capped_count > 0:
        print(f" Capped {capped_count:,} extreme values at 1st/99th percentile")
    
    # Impute remaining NaN values in ratios
    nan_count = df_with_ratios[ratio_columns].isnull().sum().sum()
    if nan_count > 0:
        imputer_ratios = SimpleImputer(strategy='median')
        df_with_ratios[ratio_columns] = imputer_ratios.fit_transform(df_with_ratios[ratio_columns])
        print(f" Imputed {nan_count:,} NaN values in ratios using median")

final_missing = df_with_ratios.isnull().sum().sum()
print(f"\n Final missing values: {final_missing:,}")

---
## Part 5: Correlation Analysis & Feature Selection

### 5.1 Analyze Correlation Among Ratios

In [ ]:
ratio_columns = list(ratios.keys())

if len(ratio_columns) >= 2:
    # Calculate correlation matrix for ratios only
    ratio_corr = df_with_ratios[ratio_columns].corr()
    
    # Visualize correlation matrix
    plt.figure(figsize=(16, 14))
    sns.heatmap(ratio_corr, annot=False, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix of Financial Ratios', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated ratio pairs (> 0.95)
    high_corr_ratios = []
    for i in range(len(ratio_corr)):
        for j in range(i+1, len(ratio_corr)):
            if abs(ratio_corr.iloc[i, j]) > 0.95:
                high_corr_ratios.append({
                    'Feature_1': ratio_corr.index[i],
                    'Feature_2': ratio_corr.columns[j],
                    'Correlation': ratio_corr.iloc[i, j]
                })
    
    if high_corr_ratios:
        high_corr_df = pd.DataFrame(high_corr_ratios).sort_values('Correlation', key=abs, ascending=False)
        print(f"\n Found {len(high_corr_ratios)} highly correlated ratio pairs (|r| > 0.95)")
        print("\nTop 10 Most Correlated Pairs:")
        print(high_corr_df.head(10).to_string(index=False))
    else:
        print("\n No highly correlated ratio pairs found (|r| > 0.95)")
else:
    print(" Not enough ratios for correlation analysis")

### 5.2 Remove Redundant Features

In [ ]:
# Build list of features to drop based on high correlation
features_to_drop = []

if len(ratio_columns) >= 2:
    for i in range(len(ratio_corr.columns)):
        for j in range(i+1, len(ratio_corr.columns)):
            if abs(ratio_corr.iloc[i, j]) > 0.95:
                feature_to_drop = ratio_corr.columns[j]
                if feature_to_drop not in features_to_drop:
                    features_to_drop.append(feature_to_drop)
                    print(f"  Dropping: {feature_to_drop} (corr={ratio_corr.iloc[i, j]:.3f} with {ratio_corr.columns[i]})")

# Create final dataset with selected features
id_columns = ['Company', 'Sector', 'Year', 'Quarter']
existing_id_cols = [col for col in id_columns if col in df_with_ratios.columns]

columns_to_keep = existing_id_cols + \
                  [col for col in ratio_columns if col not in features_to_drop]

if 'Investment_Quality' in df_with_ratios.columns:
    columns_to_keep.append('Investment_Quality')

df_final = df_with_ratios[columns_to_keep].copy()

if features_to_drop:
    print(f"\n Removed {len(features_to_drop)} redundant features")
else:
    print(f"\n No redundant features to remove")

final_feature_count = len([c for c in df_final.columns if c not in existing_id_cols + ['Investment_Quality']])
print(f" Final dataset shape: {df_final.shape}")
print(f" Final feature count: {final_feature_count}")

In [ ]:
# Display final feature list
exclude_cols = existing_id_cols + ['Investment_Quality']
final_features = [col for col in df_final.columns if col not in exclude_cols]

print(f"\nFinal Feature List ({len(final_features)} features):")
print("="*60)
for i, feature in enumerate(final_features, 1):
    print(f"{i:2d}. {feature}")

---
## Part 6: Model Training & Evaluation

### 6.1 Prepare Data for Modeling

In [ ]:
# Check if target variable exists
if 'Investment_Quality' not in df_final.columns:
    print(" ERROR: 'Investment_Quality' target variable not found!")
    print("   Cannot proceed with model training.")
    print("   Please ensure your data has an 'Investment_Quality' column with 'Good' or 'Bad' labels.")
else:
    # Separate features and target
    exclude_cols = existing_id_cols + ['Investment_Quality']
    feature_cols = [col for col in df_final.columns if col not in exclude_cols]
    
    X = df_final[feature_cols].copy()
    y = df_final['Investment_Quality'].copy()
    
    print(f" Features: {len(feature_cols)}")
    print(f" Target classes: {y.unique()}")
    print(f"\nClass distribution:")
    print(y.value_counts())
    print(f"\nClass percentages:")
    print((y.value_counts(normalize=True) * 100).round(2))
    
    # Encode target variable
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"\n Target encoding: {dict(zip(le.classes_, le.transform(le.classes_)))}")

### 6.2 Train-Test Split

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    print(f" Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
    print(f" Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
    print(f"\nTraining class distribution: {dict(zip(le.classes_, np.bincount(y_train)))}")
    print(f"Test class distribution: {dict(zip(le.classes_, np.bincount(y_test)))}")

### 6.3 Feature Scaling

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f" Features scaled using StandardScaler")
    print(f"  Training set mean: {X_train_scaled.mean():.6f}")
    print(f"  Training set std: {X_train_scaled.std():.6f}")
    print(f"  Test set mean: {X_test_scaled.mean():.6f}")
    print(f"  Test set std: {X_test_scaled.std():.6f}")

### 6.4 Train Multiple Models

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Initialize models
    class ConventionalValueModel:
        """A simple rule-based model based on standard financial benchmarks"""
        def fit(self, X, y): 
            pass
        
        def predict(self, X):
            # Simple rule: classify as 'Good' if first feature (after scaling) is positive
            # This is a simplified baseline - real conventional methods use unscaled thresholds
            return np.where(X[:, 0] > 0, 1, 0)
        
        def predict_proba(self, X):
            p = self.predict(X)
            return np.column_stack((1-p, p))
    
    models = {
        'Conventional Rule-Based': ConventionalValueModel(),
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
        'SVM (RBF)': SVC(kernel='rbf', random_state=42, probability=True),
        'Neural Network': MLPClassifier(hidden_layer_sizes=(64, 32), random_state=42, max_iter=1000)
    }
    
    print(f" Initialized {len(models)} models (including Conventional Baseline)")

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Train and evaluate all models
    results = []
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    print("\n" + "="*80)
    print("TRAINING MODELS")
    print("="*80)
    
    for name, model in models.items():
        print(f"\n[{name}]")
        print("-" * 60)
        
        # Train
        model.fit(X_train_scaled, y_train)
        print("Training completed")
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
        
        # Predictions
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else np.nan
        
        # Store results
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'ROC-AUC': roc_auc,
            'CV Mean': cv_scores.mean(),
            'CV Std': cv_scores.std()
        })
        
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1:.4f}")
        if not np.isnan(roc_auc):
            print(f"  ROC-AUC: {roc_auc:.4f}")
        print(f"  CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    
    print("\n" + "="*80)
    print("All models trained successfully!")
    print("="*80)

### 6.5 Model Comparison

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Create results dataframe
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Accuracy', ascending=False)
    
    print("\n" + "="*100)
    print("MODEL PERFORMANCE SUMMARY (sorted by Accuracy)")
    print("="*100)
    print(results_df.to_string(index=False))
    print("="*100)

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Visualize model comparison
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Accuracy comparison
    results_df_sorted = results_df.sort_values('Accuracy')
    axes[0].barh(results_df_sorted['Model'], results_df_sorted['Accuracy'], color='steelblue')
    axes[0].set_xlabel('Accuracy', fontsize=12)
    axes[0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[0].set_xlim([0, 1.05])
    axes[0].grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(results_df_sorted['Accuracy']):
        axes[0].text(v + 0.01, i, f'{v:.4f}', va='center')
    
    # Multiple metrics comparison
    metrics_df = results_df_sorted[['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score']].set_index('Model')
    metrics_df.plot(kind='barh', ax=axes[1])
    axes[1].set_xlabel('Score', fontsize=12)
    axes[1].set_title('Multiple Metrics Comparison', fontsize=14, fontweight='bold')
    axes[1].set_xlim([0, 1.05])
    axes[1].legend(loc='lower right', fontsize=10)
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 6.5.5 Comparison to Conventional Methods & Financial Implications

A key part of our analysis is comparing Machine Learning to conventional "Rule-Based" investing (e.g., Benjamin Graham style value investing).

**Key Financial Insights:**
1. **Conventional vs. ML**: Conventional rules often miss subtle non-linear relationships between ratios that ML can capture. While conventional methods are transparent, ML models often achieve higher accuracy by identifying complex risk patterns.
2. **The Cost of Errors (False Positives)**: In long-term equity investing, a **False Positive** (classifying a failing company as 'Good') is far more expensive than a **False Negative** (missing a good company). A False Positive leads to permanent capital loss, whereas a False Negative only leads to opportunity cost.
3. **Interpretation**: We focus on models that provide high **Precision** for the 'Good' class, as this minimizes the risk of buying into a bad investment.

### 6.6 Best Model Analysis

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Get best model
    best_model_name = results_df.iloc[0]['Model']
    best_model = models[best_model_name]
    
    print("\n" + "="*80)
    print("BEST MODEL ANALYSIS")
    print("="*80)
    print(f"\n Best Model: {best_model_name}")
    print(f"   Accuracy: {results_df.iloc[0]['Accuracy']:.4f}")
    print(f"   F1-Score: {results_df.iloc[0]['F1-Score']:.4f}")
    print(f"   ROC-AUC: {results_df.iloc[0]['ROC-AUC']:.4f}")
    
    # Predictions from best model
    y_pred_best = best_model.predict(X_test_scaled)
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_best)
    print("\nConfusion Matrix:")
    print(cm)
    
    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=le.classes_, yticklabels=le.classes_,
                cbar_kws={'label': 'Count'})
    plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
    plt.ylabel('Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # Classification Report
    print("\nClassification Report:")
    print("="*80)
    print(classification_report(y_test, y_pred_best, target_names=le.classes_))
    print("="*80)

### 6.7 Feature Importance (if available)

In [ ]:
if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    # Feature importance for tree-based models
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': best_model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print("\n" + "="*80)
        print("FEATURE IMPORTANCE ANALYSIS")
        print("="*80)
        print("\nTop 10 Most Important Features:")
        print(feature_importance.head(10).to_string(index=False))
        
        # Visualize
        plt.figure(figsize=(12, 8))
        top_features = feature_importance.head(15)
        plt.barh(range(len(top_features)), top_features['Importance'], color='steelblue')
        plt.yticks(range(len(top_features)), top_features['Feature'])
        plt.xlabel('Importance', fontsize=12)
        plt.title(f'Top 15 Feature Importances - {best_model_name}', fontsize=14, fontweight='bold')
        plt.gca().invert_yaxis()
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    elif hasattr(best_model, 'coef_'):
        feature_coefs = pd.DataFrame({
            'Feature': feature_cols,
            'Coefficient': best_model.coef_[0]
        }).sort_values('Coefficient', key=abs, ascending=False)
        
        print("\n" + "="*80)
        print("FEATURE COEFFICIENT ANALYSIS")
        print("="*80)
        print("\nTop 10 Features by Coefficient Magnitude:")
        print(feature_coefs.head(10).to_string(index=False))
        
        # Visualize
        plt.figure(figsize=(12, 8))
        top_features = feature_coefs.head(15)
        colors = ['green' if x > 0 else 'red' for x in top_features['Coefficient']]
        plt.barh(range(len(top_features)), top_features['Coefficient'], color=colors, alpha=0.7)
        plt.yticks(range(len(top_features)), top_features['Feature'])
        plt.xlabel('Coefficient', fontsize=12)
        plt.title(f'Top 15 Feature Coefficients - {best_model_name}', fontsize=14, fontweight='bold')
        plt.gca().invert_yaxis()
        plt.axvline(x=0, color='black', linestyle='--', linewidth=0.5)
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print("\n Feature importance/coefficients not available for this model type")

---
## Summary & Conclusions

In [ ]:
print("\n" + "="*100)
print(" "*30 + "PROJECT SUMMARY")
print("="*100)

print(f"\n Dataset:")
print(f"   Companies: {df_raw['Company'].nunique() if 'Company' in df_raw.columns else 'N/A'}")
print(f"   Time Period: {df_raw['Year'].min() if 'Year' in df_raw.columns else 'N/A'} - {df_raw['Year'].max() if 'Year' in df_raw.columns else 'N/A'}")
print(f"   Total Records: {len(df_raw):,}")
if 'Sector' in df_raw.columns:
    print(f"   Sectors: {df_raw['Sector'].nunique()}")

if len(feature_cols) > 0:
    print(f"   Final Features: {len(feature_cols)}")

print(f"\n Preprocessing:")
if 'missing_before' in locals():
    print(f"   Missing values imputed: {missing_before:,}")
if ratios:
    print(f"   Financial ratios computed: {len(ratios)}")
if features_to_drop:
    print(f"   Redundant features removed: {len(features_to_drop)}")

if 'Investment_Quality' in df_final.columns and len(feature_cols) > 0:
    print(f"\n Models:")
    print(f"   Total models evaluated: {len(models)}")
    print(f"   Best model: {best_model_name}")
    print(f"   Best accuracy: {results_df.iloc[0]['Accuracy']:.4f}")
    print(f"   Best F1-Score: {results_df.iloc[0]['F1-Score']:.4f}")
    
    conv_result = results_df[results_df['Model'] == 'Conventional Rule-Based']
    if not conv_result.empty:
        conv_acc = conv_result['Accuracy'].values[0]
        print(f"\n Conventional vs Machine Learning:")
        print(f"   Conventional Rule-Based Accuracy: {conv_acc:.4f}")
        print(f"   ML Performance Lift: {((results_df.iloc[0]['Accuracy'] / conv_acc) - 1) * 100:.1f}%")
    
    print(f"\n Financial Implications:")
    print(f"   Our best model focuses on minimizing False Positives (Permanent Capital Loss).")
    print(f"   By using non-linear combinations of {len(feature_cols)} ratios, we outperform standard benchmarks.")
    
    print(f"\n Top 3 Models:")
    for i, row in results_df.head(3).iterrows():
        print(f"   {row['Model']:30s}: Accuracy = {row['Accuracy']:.4f}, F1 = {row['F1-Score']:.4f}")

print("\n" + "="*100)
print(" "*25 + " ANALYSIS COMPLETE")
print("="*100)

---
## 🚀 How to Use This for Predicting New Companies

To predict whether a new/real company is a 'Good' or 'Bad' investment:

### Step 1: Prepare Your Data
Create a dictionary or DataFrame with the company's financial data matching the format of your training data.

### Step 2: Calculate Ratios
Apply the same ratio formulas from **Part 4** to your new data.

### Step 3: Scale & Predict
Use the `scaler` and `best_model` trained in this notebook.

```python
# Example Usage:
# new_company_data = pd.DataFrame([{
#     'Revenue': 1000,
#     'Net_Income': 100,
#     'Total_Assets': 5000,
#     # ... all other required columns
# }])

# # Calculate ratios using the same logic from Part 4
# new_ratios = calculate_ratios(new_company_data)

# # Keep only the features used in training
# new_features = new_ratios[feature_cols]

# # Scale using the fitted scaler
# new_scaled = scaler.transform(new_features)

# # Make prediction
# prediction = best_model.predict(new_scaled)
# prediction_proba = best_model.predict_proba(new_scaled)

# # Display result
# result = le.inverse_transform(prediction)[0]
# confidence = prediction_proba[0][prediction[0]]
# print(f"Prediction: {result} (Confidence: {confidence:.2%})")
```

### Important Notes:
- Ensure new data has the same columns as training data
- Handle missing values the same way (median imputation)
- Apply the same ratio calculations
- Use the same feature selection (drop redundant features)
- Scale using the fitted scaler (don't refit on new data)

---
## Export Results

In [ ]:
import os

# Create output directory if it doesn't exist
output_dir = 'analysis_outputs'
os.makedirs(output_dir, exist_ok=True)

print("Exporting results...\n")

# Save processed data with ratios
output_file = os.path.join(output_dir, 'processed_financial_data_with_ratios.csv')
df_final.to_csv(output_file, index=False)
print(f" Saved: {output_file}")

# Save model results
if 'results_df' in locals():
    output_file = os.path.join(output_dir, 'model_comparison_results.csv')
    results_df.to_csv(output_file, index=False)
    print(f" Saved: {output_file}")

# Save feature importance
if 'feature_importance' in locals():
    output_file = os.path.join(output_dir, 'feature_importance.csv')
    feature_importance.to_csv(output_file, index=False)
    print(f" Saved: {output_file}")
elif 'feature_coefs' in locals():
    output_file = os.path.join(output_dir, 'feature_coefficients.csv')
    feature_coefs.to_csv(output_file, index=False)
    print(f" Saved: {output_file}")

# Save list of features used
if 'feature_cols' in locals():
    output_file = os.path.join(output_dir, 'features_used.txt')
    with open(output_file, 'w') as f:
        f.write("Features used in final model:\n")
        f.write("=" * 50 + "\n\n")
        for i, feat in enumerate(feature_cols, 1):
            f.write(f"{i:2d}. {feat}\n")
    print(f" Saved: {output_file}")

print(f"\n All results exported to '{output_dir}/' directory!")

---
## Next Steps

### To improve this analysis:

1. **Hyperparameter Tuning**: Use GridSearchCV to optimize model parameters
2. **Feature Engineering**: Create additional derived ratios or interaction terms
3. **Ensemble Methods**: Combine multiple models for better predictions
4. **Time-Series Analysis**: Account for temporal patterns if you have multi-year data
5. **Industry-Specific Models**: Build separate models for different sectors
6. **Explainability**: Use SHAP values for better model interpretability
7. **Real-Time Data**: Integrate with financial APIs for live predictions
8. **Backtesting**: Validate predictions against actual stock performance

### Remember:
- This is a classification model, not investment advice
- Always validate predictions with fundamental analysis
- Markets are complex - no model is perfect
- Past performance doesn't guarantee future results